In [ ]:
import pandas as pd
import numpy as np
import json
from collections import Counter
import copy

In [ ]:
commit_sharings = "./20231012_230826_commit_sharings.json"
pull_request_sharings = "./20231012_233628_pr_sharings.json"
issue_sharings = "./20231012_235128_issue_sharings.json"
file_sharings = "./20231012_234250_file_sharings.json"
discussion_sharings = "./20231012_235320_discussion_sharings.json"

path_list_synonym = "./synonyms_for_normalization.txt"

In [ ]:
with open(commit_sharings, 'r') as f:
    json1 = json.load(f)

# Abra o arquivo json
with open(pull_request_sharings, 'r') as f:
    json2 = json.load(f)

# Abra o arquivo json
with open(issue_sharings, 'r') as f:
    json3 = json.load(f)

# Abra o arquivo json
with open(file_sharings, 'r') as f:
    json4 = json.load(f)

# Abra o arquivo json
with open(discussion_sharings, 'r') as f:
    json5 = json.load(f)

jsons = {
    "data1": json1,
    "data2": json2,
    "data3": json3,
    "data4": json4,
    "data5": json5
}

# Normalisation

In [ ]:
with open(path_list_synonym, 'r') as file:
    synonyms = {}

    for line in file:
        key, value = line.strip().split('=')
        synonyms[key.strip()] = value.strip().split(',')

In [ ]:

for DataTable in jsons:
  for elements in jsons[DataTable]["Sources"]:
    if elements["RepoLanguage"] == None :
      elements["RepoLanguage"] = "none"
    else:
      for language, alias in synonyms.items():
        if elements["RepoLanguage"] in alias:
          elements["RepoLanguage"] = language

for DataTable in jsons:
  for elements in jsons[DataTable]["Sources"]:
    for element in elements["ChatgptSharing"]:
      if "Conversations" in element and element["Conversations"]:
        for conversation in element["Conversations"]:
          if "ListOfCode" in conversation and conversation["ListOfCode"]:
            for snipetType in conversation["ListOfCode"]:
              if snipetType["Type"] == None :
                snipetType["Type"] = "none"
              else:
                for language, alias in synonyms.items():
                  if  snipetType["Type"] in alias:
                    snipetType["Type"] = language
    


In [ ]:

# file_path = './normalised/multifile_normalised.json'

# with open(file_path, 'w') as file:
#   json.dump(jsons, file)

# Data Extraction

In [ ]:
repoLang = []
for DataTable in jsons:
  for elements in jsons[DataTable]["Sources"]:
    repoLang.append(elements["RepoLanguage"])


In [ ]:
counting = Counter(repoLang)

countingDict = dict(counting)

In [ ]:
df0 = pd.DataFrame(list(countingDict.items()), columns=['Language', 'Count'])

In [ ]:
repoByLanguage = {}

for key in counting.keys():
  repoByLanguage.setdefault(key, [])
  for DataTable in jsons:
    for elements in jsons[DataTable]["Sources"]:
      if elements["RepoLanguage"] == key:
        repoByLanguage[key].append(elements["RepoName"])


In [ ]:
df1 = pd.DataFrame(list(repoByLanguage.items()), columns=['Language', 'Repos'])

In [ ]:
repoByLanguageCount = copy.deepcopy(repoByLanguage)

for key, alias in repoByLanguage.items():
    repoByLanguage[key] = list(set(alias))
    repoByLanguageCount[key] = len(list(set(alias)))

In [ ]:
df2 = pd.DataFrame(list(repoByLanguage.items()), columns=['Language', 'UniqueRepos'])
df3 = pd.DataFrame(list(repoByLanguageCount.items()), columns=['Language', 'UniqueReposCount'])

## CountChat

In [ ]:
chatByLanguage = {}
for DataTable in jsons:
  for elements in jsons[DataTable]["Sources"]:
    chatByLanguage.setdefault(elements["RepoLanguage"], [])
    for element in elements["ChatgptSharing"]:
      chatByLanguage[elements["RepoLanguage"]].append(element["URL"])



In [ ]:
df4 = pd.DataFrame(list(chatByLanguage.items()), columns=['Language', 'Chat'])

In [ ]:
chatByLanguageCount = copy.deepcopy(chatByLanguage)
for key, alias in chatByLanguage.items():
    chatByLanguage[key] = list(set(alias))
    chatByLanguageCount[key] = len(list(set(alias)))

In [ ]:
chatByLanguage
df5 = pd.DataFrame(list(chatByLanguage.items()), columns=['Language', 'UniqueChat'])
df6 = pd.DataFrame(list(chatByLanguageCount.items()), columns=['Language', 'UniqueChatCount'])

## Code Snipets

In [ ]:
snipetsByLanguage = {}
for DataTable in jsons:
  for elements in jsons[DataTable]["Sources"]:
    snipetsByLanguage.setdefault(elements["RepoLanguage"], [])
    for element in elements["ChatgptSharing"]:
      if "Conversations" in element and element["Conversations"]:
        for conversation in element["Conversations"]:
          if "ListOfCode" in conversation and conversation["ListOfCode"]:
            for snipetType in conversation["ListOfCode"]:
              snipetsByLanguage[elements["RepoLanguage"]].append(snipetType["Type"])



In [ ]:
snipetsByLanguage
snipetsByLanguagePercentage = copy.deepcopy(snipetsByLanguage)
snipetsByLanguageTotal = copy.deepcopy(snipetsByLanguage)

In [ ]:

for key, alias in snipetsByLanguage.items():
    total_elements_on_lista = len(alias)

    count_elements = Counter(alias)

    snipetsByLanguagePercentage[key] = {element: f'{(ocurrency / total_elements_on_lista) * 100:.3f}' for element, ocurrency in count_elements.items()}
    snipetsByLanguage[key] = dict(Counter(snipetsByLanguage[key]))
    snipetsByLanguageTotal[key] = (sum(Counter(snipetsByLanguage[key]).values()))

In [ ]:
df7 = pd.DataFrame(list(snipetsByLanguage.items()), columns=['Language', 'SnippetLang'])

df8 = pd.DataFrame(list(snipetsByLanguageTotal.items()), columns=['Language', 'SnippetLangCount'])

df9 = pd.DataFrame(list(snipetsByLanguagePercentage.items()), columns=['Language', 'SnippetLangPercentage'])

final_df = pd.merge(df0, df1, on='Language').merge(df2, on='Language').merge(df3, on='Language').merge(df4, on='Language')
final_df = final_df.merge(df5, on='Language').merge(df6, on='Language').merge(df7, on='Language').merge(df8, on='Language').merge(df9, on='Language')
final_df.to_csv('results/merged_results_table.csv', index=False)

## Verify Snipets by language

In [ ]:

rows = []
for columns, values in snipetsByLanguage.items():
    row = {'language': columns}
    row.update(values)
    rows.append(row)

snipetsByLanguageDF = pd.DataFrame(rows)
snipetsByLanguageDF = snipetsByLanguageDF.T


snipetsByLanguageDF = snipetsByLanguageDF.fillna(0)
snipetsByLanguageDF.to_csv('results/merged_snipetsByLanguageDF.csv')

## Snipets By Language in Percentage

In [ ]:
rows = []
for columns, values in snipetsByLanguagePercentage.items():
    row = {'language': columns}
    row.update(values)
    rows.append(row)


snipetsByLanguagePercentageDF = pd.DataFrame(rows)
snipetsByLanguagePercentageDF = snipetsByLanguagePercentageDF.T


snipetsByLanguagePercentageDF = snipetsByLanguagePercentageDF.fillna(0)
snipetsByLanguagePercentageDF.to_csv('results/merged_snipetsByLanguagePercentageDF.csv')